<a href="https://colab.research.google.com/github/SarahPendhari/ATS-Resume-Parser-NLP/blob/main/FINAL_ATS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio transformers datasets PyPDF2 python-docx -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 635.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import gradio as gr
from PyPDF2 import PdfReader
import docx
import re
import numpy as np
import traceback

In [3]:
# Load and preprocess the dataset
df = pd.read_csv('data.csv')
df['text'] = df['Job Description'] + ' ' + df['company'] + ' ' + df['position']

In [4]:
# Create a binary label (this is an example, adjust based on your specific task)
df['label'] = np.random.randint(0, 2, df.shape[0])

In [5]:
# Split the dataset
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)

In [6]:
# Tokenize the text
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
def tokenize_function(examples):
    tokenized = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)
    tokenized['labels'] = examples['label']
    return tokenized

In [8]:
# Prepare the datasets
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

In [9]:
# Tokenize the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=train_dataset.column_names)
eval_dataset = eval_dataset.map(tokenize_function, batched=True, remove_columns=eval_dataset.column_names)

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [10]:
# Set device to cuda if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
# Fine-tune DistilBERT
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2).to(device)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

trainer.train()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.683373
2,No log,0.685988
3,No log,0.689716


TrainOutput(global_step=24, training_loss=0.6970287164052328, metrics={'train_runtime': 28.6902, 'train_samples_per_second': 13.071, 'train_steps_per_second': 0.837, 'total_flos': 49675274496000.0, 'train_loss': 0.6970287164052328, 'epoch': 3.0})

In [12]:
# Function to extract text from PDF
def extract_text_from_pdf(file):
    pdf = PdfReader(file)
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
    return text

In [13]:
# Function to extract text from DOCX
def extract_text_from_docx(file):
    doc = docx.Document(file)
    text = ""
    for para in doc.paragraphs:
        text += para.text + "\n"
    return text

In [14]:
# Function to calculate ATS score
def calculate_ats_score(resume_text, job_description):
    # Tokenize and get embeddings for resume and job description
    resume_encoding = tokenizer(resume_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    job_encoding = tokenizer(job_description, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)

    # Get model outputs
    with torch.no_grad():
        resume_output = model(**resume_encoding)
        job_output = model(**job_encoding)

    # The logits are expected to be of shape [batch_size, num_labels]
    # We use the logits to represent the output embeddings
    resume_embedding = resume_output.logits  # shape [1, num_labels]
    job_embedding = job_output.logits  # shape [1, num_labels]

    # Make sure the embeddings are the same size and compatible for comparison
    if resume_embedding.size() != job_embedding.size():
        raise ValueError("Embedding sizes don't match")

    # Calculate similarity score using cosine similarity
    similarity = torch.nn.functional.cosine_similarity(resume_embedding, job_embedding, dim=1)

    return similarity.item() * 100  # Convert to percentage

In [15]:
# Function to suggest improvements
def suggest_improvements(resume_text, job_description):
    # Extract keywords from job description
    job_keywords = set(re.findall(r'\b\w+\b', job_description.lower()))
    resume_keywords = set(re.findall(r'\b\w+\b', resume_text.lower()))

    missing_keywords = job_keywords - resume_keywords

    suggestions = []
    if missing_keywords:
        suggestions.append(f"Consider adding these keywords: {', '.join(list(missing_keywords)[:10])}")

    return suggestions


In [16]:
# Gradio interface
def process_resume(resume_file, job_description):
    try:
        if resume_file is None:
            return "Error: No file uploaded. Please upload a PDF or DOCX file."

        # Read file content directly
        if resume_file.name.endswith('.pdf'):
            resume_text = extract_text_from_pdf(resume_file)
        elif resume_file.name.endswith('.docx'):
            resume_text = extract_text_from_docx(resume_file)
        else:
            return "Error: Unsupported file format. Please upload a PDF or DOCX file."

        if not resume_text.strip():
            return "Error: Unable to extract text from the uploaded file. Please ensure the file is not empty or corrupted."

        if not job_description.strip():
            return "Error: Job description is empty. Please provide a job description."

        ats_score = calculate_ats_score(resume_text, job_description)
        suggestions = suggest_improvements(resume_text, job_description)

        result = f"ATS Score: {ats_score:.2f}%\n\nAreas for Improvement:\n"
        result += "\n".join(suggestions)

        return result
    except Exception as e:
        error_msg = f"An error occurred: {str(e)}\n\nTraceback:\n{traceback.format_exc()}"
        print(error_msg)  # This will print the full error message in your console/logs
        return f"Error: An unexpected error occurred. Please try again or contact support if the issue persists.\nDetails: {str(e)}"


In [17]:
# Gradio interface
iface = gr.Interface(
    fn=process_resume,
    inputs=[
        gr.File(label="Upload Resume (PDF or DOCX)"),
        gr.Textbox(label="Job Description")
    ],
    outputs=gr.Textbox(label="ATS Analysis Result"),
    title="ATS Resume Parser",
    description="Upload your resume and paste the job description to get an ATS analysis."
)

iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://96cf3e06cdb30f8edf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
